# Schudeler Query from Bronce to Silver for Images Items

Library

In [1]:
import pandas as pd
from pandas_gbq import read_gbq, to_gbq
import json

Project Info

In [2]:
project_id = "big-data-lol"
dataset_id_from = "Bronze_LOL_Dataset"
dataset_id_to = "Silver_LOL_Dataset"
table_id_from = "bronze_batch_img_item"
table_id_to = "silver_batch_img_item"

Lol current patch

In [3]:
current_patch = "14.12"

Query

In [4]:
query = f'''
SELECT *
FROM {project_id}.{dataset_id_from}.{table_id_from}
WHERE TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) = TIMESTAMP("2024-06-17") AND
key = "image"


'''

Query in BigQuery Dataset

In [5]:
df = read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [6]:
df.head(1)

,UID,Key,Value,FileName
0,cce8c94b-1f85-4e4c-aa64-4a90831f78be,image,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",223031.json


Edit df

In [7]:
# Transformar el DataFrame
df['Key'] = df['FileName'].str.replace('.json', '')  # Extraer el nombre del archivo sin extensión
df.rename(columns={'Value': 'image'}, inplace=True)  # Renombrar la columna 'Value' a 'image'
df['patch'] = current_patch  # Añadir la columna 'patch'

# Seleccionar y reordenar las columnas deseadas
df = df[['UID', 'Key', 'image', 'patch']]

In [8]:
df.head(2)

,UID,Key,image,patch
0,cce8c94b-1f85-4e4c-aa64-4a90831f78be,223031,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",14.12
1,5e6c5980-8583-4256-a925-7f371fe4d4ab,227031,"""iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAIAAAAlC+aJA...",14.12


Schema create

In [9]:
from google.cloud import bigquery
schema = [

  bigquery.SchemaField('id', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('key', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('image', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('patch', 'STRING', 'NULLABLE')

]



In [10]:
client = bigquery.Client(project=project_id)

validated_path = '/tmp/validated_' + "champs_to_silver.csv"
# se transforma el df a csv
df.to_csv(validated_path, index=False)

print(validated_path)
# Cargar los datos validados en BigQuery
dataset_ref = client.dataset(dataset_id_to)
table_ref = dataset_ref.table(table_id_to)

#configuracion del job
job_config = bigquery.LoadJobConfig()

job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.schema = schema
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


with open(validated_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()


/tmp/validated_champs_to_silver.csv


LoadJob<project=big-data-lol, location=US, id=e449c6bc-bb0e-4644-a1be-302dd1b06fd4>

Job Configuration